# Association analysis sample (Online Retail II)

- Online Retail II Data Set https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

In [1]:
import pandas as pd
import dask.dataframe as dd
import dask

/usr/local/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
# Dataset
_read_xlsx_delayed = dask.delayed(pd.read_excel)(
    'online_retail_II.xlsx'
)
DF_RAW = dd.from_delayed(_read_xlsx_delayed).compute()

In [3]:
display(DF_RAW.shape[0])
display(DF_RAW.head())

525461

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
display(DF_RAW.isnull().sum())

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [5]:
print(
    DF_RAW['Country'].value_counts().head(15)
)

Country
United Kingdom     485852
EIRE                 9670
Germany              8129
France               5772
Netherlands          2769
Spain                1278
Switzerland          1187
Portugal             1101
Belgium              1054
Channel Islands       906
Sweden                902
Italy                 731
Australia             654
Cyprus                554
Austria               537
Name: count, dtype: int64


In [6]:
# Data preprocessing.
df = DF_RAW.copy()

df['order_type'] = df['Invoice'].map(
    lambda x: str(x)[0]
)

display(df.head(3))
display(df.tail(3))
display(df['order_type'].value_counts())

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,order_type
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,4
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,4


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,order_type
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,5
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,5
525460,538171,21931,JUMBO STORAGE BAG SUKI,2,2010-12-09 20:01:00,1.95,17530.0,United Kingdom,5


order_type
5    406763
4    108489
C     10206
A         3
Name: count, dtype: int64

In [7]:
# Limit new orders only.
df = df[
    df['order_type'].isin(['5', '4'])
]

display(df['order_type'].value_counts())
display(df.shape[0])

order_type
5    406763
4    108489
Name: count, dtype: int64

515252

In [8]:
df = df[
    df['Country'] == 'Germany'
]

display(df.shape[0])

7661

In [9]:
# Convert vertically arranged data to horizontally.
qt_groupeby_invoice_and_stockcode = df.groupby(
    ['Invoice', 'StockCode']
)['Quantity'].sum()

display(qt_groupeby_invoice_and_stockcode.head())

Invoice  StockCode
489526   20676         8
         20682         6
         20718        10
         20914        12
         20964         3
Name: Quantity, dtype: int64

In [10]:
moved_stockcode_to_column = \
    qt_groupeby_invoice_and_stockcode.unstack().reset_index().fillna(0).set_index('Invoice')

display(moved_stockcode_to_column.shape)
display(moved_stockcode_to_column.head())

(347, 1457)

StockCode,10002,10125,10135,11001,15034,15036,15039,16012,16016,16033,...,85232A,85232B,85232D,90018A,90019A,90200E,ADJUST,M,PADS,POST
Invoice,,,,,,,,,,,,,,,,,,,,,
489526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
490395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
490563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
490564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
490682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [11]:
basket_df = moved_stockcode_to_column.apply(lambda x: x > 0)

display(basket_df.head())

StockCode,10002,10125,10135,11001,15034,15036,15039,16012,16016,16033,...,85232A,85232B,85232D,90018A,90019A,90200E,ADJUST,M,PADS,POST
Invoice,,,,,,,,,,,,,,,,,,,,,
489526,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
490395,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
490563,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
490564,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
490682,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [12]:
# Prepare items name dictionary.
stockcodes_and_descriptions = df[
    ['StockCode', 'Description']
].drop_duplicates()

stockcodes_and_descriptions['StockCode'] = stockcodes_and_descriptions['StockCode'].astype('str')
stockcodes_and_descriptions['Description'] = stockcodes_and_descriptions['Description'].astype('str')

stockcodes_and_descriptions = stockcodes_and_descriptions.set_index('StockCode')

display(stockcodes_and_descriptions.head())
display(stockcodes_and_descriptions.loc[
    stockcodes_and_descriptions.index[:10]
])

,Description
StockCode,
85049E,SCANDINAVIAN REDS RIBBONS
21976,PACK OF 60 MUSHROOM CAKE CASES
21498,RED SPOTS WRAP
22077,6 RIBBONS RUSTIC CHARM
84946,ANTIQUE SILVER TEA GLASS ETCHED


,Description
StockCode,
85049E,SCANDINAVIAN REDS RIBBONS
21976,PACK OF 60 MUSHROOM CAKE CASES
21498,RED SPOTS WRAP
21498,RED RETROSPOT WRAP
22077,6 RIBBONS RUSTIC CHARM
84946,ANTIQUE SILVER TEA GLASS ETCHED
84948,SILVER HANGING T-LIGHT HOLDER DOME
21537,RETRO SPOTS PUDDING BOWL
21537,RED RETROSPOT PUDDING BOWL


There are 2 different Description for the same product, and the policy is to select the product with the longest string.

In [13]:
merge_different_desc_items = stockcodes_and_descriptions.copy()

merge_different_desc_items = merge_different_desc_items[
    merge_different_desc_items['Description'].map(
        lambda x: x.isupper()
    )
]

merge_different_desc_items['characters'] = merge_different_desc_items['Description'].map(len)
merge_different_desc_items = merge_different_desc_items.sort_values(
    ['StockCode', 'characters'],
    ascending=[True, False]
)


In [14]:
# Prepare association analysis
from mlxtend.frequent_patterns import apriori, association_rules

In [15]:
# Apriori algorithm analysis
highly_support_items = apriori(
    basket_df,
    min_support=0.07,
    use_colnames=True
)

print(highly_support_items.shape[0])

display(
    highly_support_items.sort_values(
        'support',
        ascending=False
    ).head(10)
)

90


,support,itemsets
46,0.780980,(POST)
29,0.288184,(22326)
74,0.265130,"(POST, 22326)"
3,0.216138,(20719)
30,0.193084,(22328)
53,0.193084,"(POST, 20719)"
75,0.181556,"(22328, POST)"
22,0.178674,(21731)
12,0.155620,(21238)
68,0.155620,"(POST, 21731)"


In [16]:
# Extract association rules
association_rules = association_rules(
    highly_support_items,
    metric="lift",
    min_threshold=1
)
association_rules = association_rules.sort_values(
    "lift",
    ascending=False
).reset_index(drop=True)

print(highly_support_items.shape[0])

display(association_rules.head(10))

90


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(21238),(21242),0.155620,0.106628,0.089337,0.574074,5.383884,0.072744,2.097482,0.964329
1,(21242),(21238),0.106628,0.155620,0.089337,0.837838,5.383884,0.072744,5.207012,0.911446
2,"(POST, 21238)",(20676),0.123919,0.109510,0.072046,0.581395,5.309058,0.058476,2.127281,0.926447
3,(20676),"(POST, 21238)",0.109510,0.123919,0.072046,0.657895,5.309058,0.058476,2.560851,0.911456
4,(21240),(21238),0.092219,0.155620,0.074928,0.812500,5.221065,0.060577,4.503362,0.890598
5,(21238),(21240),0.155620,0.092219,0.074928,0.481481,5.221065,0.060577,1.750720,0.957469
6,(21238),(20676),0.155620,0.109510,0.086455,0.555556,5.073099,0.069413,2.003602,0.950853
7,(20676),(21238),0.109510,0.155620,0.086455,0.789474,5.073099,0.069413,4.010807,0.901618
8,(21238),"(POST, 20676)",0.155620,0.092219,0.072046,0.462963,5.020255,0.057695,1.690351,0.948396
9,"(POST, 20676)",(21238),0.092219,0.155620,0.072046,0.781250,5.020255,0.057695,3.860025,0.882159


In [17]:
highly_relevantive_items = [21976, 21238, 21242, 20676, 21240]

for item in highly_relevantive_items:
    print(
        item,
        stockcodes_and_descriptions.loc[str(item)],
        "\n"
    )

21976 Description    PACK OF 60 MUSHROOM CAKE CASES
Name: 21976, dtype: object 

21238                  Description
StockCode                   
21238         RED SPOTTY CUP
21238      RED RETROSPOT CUP 

21242                     Description
StockCode                      
21242         RED SPOTTY PLATE 
21242      RED RETROSPOT PLATE  

20676                   Description
StockCode                    
20676         RED SPOTTY BOWL
20676      RED RETROSPOT BOWL 

21240                  Description
StockCode                   
21240       BLUE  SPOTTY CUP
21240      BLUE POLKADOT CUP 

